In [1]:
# Needed to switch directory in Google drive so as to import MARL env.

from google.colab import drive 
drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
#%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/gym_continuousDoubleAuction/"

!pwd

!pip install -r requirements.txt

!pip show tensorflow
!pip show ray

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
Obtaining gym_continuousDoubleAuction from git+https://github.com/ChuaCheowHuan/gym-continuousDoubleAuction.git@c897137cbcc93ca71cbd51c27e683c3298f6562d#egg=gym_continuousDoubleAuction (from -r requirements.txt (line 24))
  Skipping because already up-to-date.
  Found existing installation: gym-continuousDoubleAuction 0.0.1
    Can't uninstall 'gym-continuousDoubleAuction'. No files were found to uninstall.
  Running setup.py develop for gym-continuousDoubleAuction
Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.

In [0]:
#CDA_env_disc_RLlib

# https://github.com/ray-project/ray/blob/master/python/ray/rllib/examples/multiagent_cartpole.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
"""Simple example of setting up a multi-agent policy mapping.
Control the number of agents and policies via --num-agents and --num-policies.
This works with hundreds of agents and policies, but note that initializing
many TF policies will take some time.
Also, TF evals might slow down with large numbers of policies. To debug TF
execution, set the TF_TIMELINE_DIR environment variable.
"""

# rllib rollout ~/ray_results/PPO/PPO_MMenv-v0_0_2019-08-09_00-06-10t4b1lscr/checkpoint-1 --run PPO --env MMenv-v0 --steps 100

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse
import gym
import random
import numpy as np

import ray
from ray import tune
from ray.rllib.models import Model, ModelCatalog

from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

from ray.tune.registry import register_env
from ray.rllib.utils import try_import_tf


from ray.rllib.policy.policy import Policy
#from ray.rllib.policy.tf_policy import TFPolicy
#from ray.rllib.policy import Policy


#from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG


import sys
if "../" not in sys.path:
    sys.path.append("../")
#from exchg.x.y import z

#from envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv
from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

tf = try_import_tf()

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--num-agents", type=int, default=4)
parser.add_argument("--num-policies", type=int, default=4)
parser.add_argument("--num-iters", type=int, default=10)
parser.add_argument("--simple", action="store_true")

_StoreTrueAction(option_strings=['--simple'], dest='simple', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [0]:
class CustomModel_1(Model):
    def _lstm(self, Inputs, cell_size):
        s = tf.expand_dims(Inputs, axis=1, name='time_major')  # [time_step, feature] => [time_step, batch, feature]
        lstm_cell = tf.nn.rnn_cell.LSTMCell(cell_size)
        self.init_state = lstm_cell.zero_state(batch_size=1, dtype=tf.float32)
        # time_major means [time_step, batch, feature] while batch major means [batch, time_step, feature]
        outputs, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=s, initial_state=self.init_state, time_major=True)
        lstm_out = tf.reshape(outputs, [-1, cell_size], name='flatten_rnn_outputs')  # joined state representation
        return lstm_out

    def _build_layers_v2(self, input_dict, num_outputs, options):
        hidden = 512
        cell_size = 256
        #S = input_dict["obs"]
        S = tf.layers.flatten(input_dict["obs"])
        with tf.variable_scope(tf.VariableScope(tf.AUTO_REUSE, "shared"),
                               reuse=tf.AUTO_REUSE,
                               auxiliary_name_scope=False):
            last_layer = tf.layers.dense(S, hidden, activation=tf.nn.relu, name="fc1")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc2")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc3")

        last_layer = self._lstm(last_layer, cell_size)

        output = tf.layers.dense(last_layer, num_outputs, activation=tf.nn.softmax, name="mu")

        return output, last_layer

In [0]:
class CustomModel(TFModelV2):
    """Example of a custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        
        super(CustomModel, self).__init__(obs_space, action_space, num_outputs, model_config, name)
        
        self.model = FullyConnectedNetwork(obs_space, action_space, num_outputs, model_config, name)
        #print('obs_space', obs_space)

        self.register_variables(self.model.variables())

    def forward(self, input_dict, state, seq_lens):
        return self.model.forward(input_dict, state, seq_lens)

    def value_function(self):
        return self.model.value_function()

In [0]:
def make_RandomPolicy(_seed):

    # a hand-coded policy that acts at random in the env (doesn't learn)
    class RandomPolicy(Policy):
        """Hand-coded policy that returns random actions."""
        def __init__(self, observation_space, action_space, config):
            self.observation_space = observation_space
            self.action_space = action_space
            self.action_space.seed(_seed)

        def compute_actions(self,
                            obs_batch,
                            state_batches,
                            prev_action_batch=None,
                            prev_reward_batch=None,
                            info_batch=None,
                            episodes=None,
                            **kwargs):
            """Compute actions on a batch of observations."""
            return [self.action_space.sample() for _ in obs_batch], [], {}

        def learn_on_batch(self, samples):
            """No learning."""
            #return {}
            pass

        def get_weights(self):
            pass

        def set_weights(self, weights):
            pass

    return RandomPolicy

In [7]:
#args = parser.parse_args()

# set log_to_driver=False to off render output so as to prevent browser from hanging.
#ray.init(ignore_reinit_error=True)
#ray.init(ignore_reinit_error=True, num_cpus=2)
#ray.init(ignore_reinit_error=True, webui_host='127.0.0.1', num_cpus=2)
ray.init(ignore_reinit_error=True, log_to_driver=False, webui_host='127.0.0.1', num_cpus=2)

2020-03-17 07:28:49,653	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-17 07:28:49,655	INFO resource_spec.py:212 -- Starting Ray with 6.74 GiB memory available for workers and up to 3.38 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-17 07:28:50,156	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2020-03-17_07-28-49_652205_2354/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-17_07-28-49_652205_2354/sockets/raylet',
 'redis_address': '172.28.0.2:42788',
 'session_dir': '/tmp/ray/session_2020-03-17_07-28-49_652205_2354',
 'webui_url': '127.0.0.1:8265'}

In [0]:
num_agents = 4
num_policies = num_agents
num_iters = 3
simple = False #store_true
#num_of_traders = args.num_agents
num_of_traders = num_agents
tape_display_length = 10 #100
tick_size = 1
init_cash = 1000000
max_step = 700 # per episode #700
episode = 5 #9

In [9]:
single_CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step)
obs_space = single_CDA_env.observation_space
act_space = single_CDA_env.action_space
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step))
ModelCatalog.register_custom_model("model_disc", CustomModel_1)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
# Each policy can have a different configuration (including custom model)
def gen_policy(i):
    config = {"model": {"custom_model": "model_disc"},
              "gamma": 0.99,}
    return (None, obs_space, act_space, config)

In [0]:
def policy_mapper(agent_id):
    for i in range(num_agents):
        if agent_id == i:
            return "policy_{}".format(i)

In [0]:
# Setup PPO with an ensemble of `num_policies` different policies

# Dictionary of policies
#policies = {"policy_{}".format(i): gen_policy(i) for i in range(args.num_policies)}
policies = {"policy_{}".format(i): gen_policy(i) for i in range(num_policies)}

In [13]:
# override policy with random policy

def set_RandomPolicy(policies):
    """
    Set 1st policy as PPO & override all other policies as RandomPolicy with
    different seed.
    """

    for i in range(num_agents):
        if i == num_agents-1:
            break
        x = i + 1
        policies["policy_{}".format(num_policies-x)] = (make_RandomPolicy(num_policies-x), obs_space, act_space, {})

    print('policies:', policies)
    return 0

set_RandomPolicy(policies)

policy_ids = list(policies.keys())

policies: {'policy_0': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_2': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_3': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {})}


In [14]:
tune.run(#PPOTrainer,
          "PPO",
          #"PG",
          #queue_trials=False,
          #resources_per_trial={"cpu": 2,
          #                     "gpu": 0},

          #stop={"training_iteration": args.num_iters},
          #stop={"timesteps_total": max_step,
          #      "training_iteration": num_iters},
          stop={"timesteps_total": max_step * episode},

          config={"env": "continuousDoubleAuction-v0",

                  #"log_level": "DEBUG",
                  #"simple_optimizer": args.simple,
                  #"simple_optimizer": True,
                  #"num_sgd_iter": 10,

                  #"gamma": 0.9,

                  # Number of rollout worker actors to create for parallel sampling.
                  # Setting to 0 will force rollouts to be done in the trainer actor.
                  "num_workers": 1, # Colab (only 2 CPUs or 1 GPU) #1
                  "num_envs_per_worker": 2, #2

                  #"timesteps_per_iteration": max_step,

                  # https://github.com/ray-project/ray/issues/4628
                  "sample_batch_size": 32, # number of environment steps sampled from each environment
                  "train_batch_size": 128, # minibatch size must be >= 128, number of environment steps sampled from all available environments

                  "multiagent": {"policies_to_train": ["policy_0"],
                                 "policies": policies,
                                 #"policy_mapping_fn": tune.function(lambda agent_id: random.choice(policy_ids)),
                                 #"policy_mapping_fn": tune.function(policy_mapper),
                                 "policy_mapping_fn": policy_mapper,
                                },
                  },
          )

#ray.shutdown()

Trial name,status,loc
PPO_continuousDoubleAuction-v0_f2ebbe2a,RUNNING,


Result for PPO_continuousDoubleAuction-v0_f2ebbe2a:
  custom_metrics: {}
  date: 2020-03-17_07-29-29
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 5863ef4ca418454cb449ce9f78f4c8ba
  experiment_tag: '0'
  hostname: 9e785dd5f6c5
  info:
    grad_time_ms: 14915.606
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 7.934357643127441
        entropy_coeff: 0.0
        kl: 0.015403380617499352
        policy_loss: -0.12289988994598389
        total_loss: 268038784.0
        vf_explained_var: 0.0
        vf_loss: 268038784.0
    load_time_ms: 120.425
    num_steps_sampled: 128
    num_steps_trained: 128
    sample_time_ms: 3176.368
    update_time_ms: 1529.422
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_p

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_f2ebbe2a,RUNNING,172.28.0.2:2443,nan,19.8715,128,1


Result for PPO_continuousDoubleAuction-v0_f2ebbe2a:
  custom_metrics: {}
  date: 2020-03-17_07-29-46
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 5863ef4ca418454cb449ce9f78f4c8ba
  experiment_tag: '0'
  hostname: 9e785dd5f6c5
  info:
    grad_time_ms: 14465.062
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 7.939593315124512
        entropy_coeff: 0.0
        kl: 0.01406663004308939
        policy_loss: -0.07625341415405273
        total_loss: 16426016.0
        vf_explained_var: 0.0
        vf_loss: 16426018.0
    load_time_ms: 60.873
    num_steps_sampled: 256
    num_steps_trained: 256
    sample_time_ms: 3202.707
    update_time_ms: 777.485
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percen

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_f2ebbe2a,RUNNING,172.28.0.2:2443,nan,37.1452,256,2


...

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_f2ebbe2a,RUNNING,172.28.0.2:2443,0,463.417,3456,27


Result for PPO_continuousDoubleAuction-v0_f2ebbe2a:
  custom_metrics: {}
  date: 2020-03-17_07-37-10
  done: true
  episode_len_mean: 701.0
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 0
  episodes_total: 4
  experiment_id: 5863ef4ca418454cb449ce9f78f4c8ba
  experiment_tag: '0'
  hostname: 9e785dd5f6c5
  info:
    grad_time_ms: 13342.175
    learner:
      policy_0:
        cur_kl_coeff: 0.675000011920929
        cur_lr: 4.999999873689376e-05
        entropy: 8.05618667602539
        entropy_coeff: 0.0
        kl: 0.013894252479076385
        policy_loss: -0.08181355893611908
        total_loss: 9196808.0
        vf_explained_var: 0.0
        vf_loss: 9196808.0
    load_time_ms: 1.167
    num_steps_sampled: 3584
    num_steps_trained: 3584
    sample_time_ms: 3229.931
    update_time_ms: 28.198
  iterations_since_restore: 28
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 89.

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_f2ebbe2a,TERMINATED,,0,479.909,3584,28


Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_f2ebbe2a,TERMINATED,,0,479.909,3584,28


2020-03-17 07:37:10,440	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.
